In [2]:
import spacy
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from xml.etree import cElementTree as ET
import re
import json
from collections import Counter
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from analysis_code.analyze_corpus import *

In [3]:
example = '/home/luluvyd/corpus/gutenberg-poetry-example.ndjson'
infile = '/home/luluvyd/corpus/gutenberg-poetry-v001.ndjson'

outdirect = '/home/luluvyd/corpus/gutenberg-break/poetry'

In [6]:
def breakFileSmart(infile, N = 20000): 
    '''
    break a giant input file into small output files of size n articles 
    parameters : infile : file handler of a giant input file
    i : i-th input file
    n : number of articles in each output file
    '''
    line = infile.readline()
    buffer = ""
    curr = ""
    j = 0 
    pgid = -1
    
    while line:
        res = json.loads(line) 
        c = res['s']
        cgid = res['gid']
        if (cgid != pgid): 
            pgid = cgid
            if (len(curr) + len(buffer) >= N): 
                write_file = outdirect + '-' + str(j) + '.txt'
                with open(write_file, "w") as outfile:
                    outfile.write(buffer) 
                outfile.close() 
                buffer = curr 
                j += 1
            else:                 
                buffer += curr
                curr = ""
        curr += c + ' '
        line = infile.readline()
    if buffer: 
        write_file = outdirect + '-' + str(j) + '.txt'
        with open(write_file, "w") as outfile:
            outfile.write(buffer)
        outfile.close()         

def breakFile(infile, N = 15000): 
    '''
    break a giant input file into small output files of size n articles 
    parameters : infile : file handler of a giant input file
    i : i-th input file
    n : number of articles in each output file
    '''
    line = infile.readline()
    buffer = ""
    curr = ""
    j = 0 
    i = 0
    
    while line:
        res = json.loads(line) 
        c = res['s']
        cgid = res['gid']
        i += 1
        if (i % N == 0): 
            write_file = outdirect + '-' + str(j) + '.txt'
            with open(write_file, "w") as outfile:
                outfile.write(buffer) 
            outfile.close() 
            buffer = ""
            j += 1 
            print(i)
        buffer += c + ' '
        line = infile.readline()
    if buffer: 
        write_file = outdirect + '-' + str(j) + '.txt'
        with open(write_file, "w") as outfile:
            outfile.write(buffer)
        outfile.close()       
        
def countFileSize(infile): 
    '''
    break a giant input file into small output files of size n articles 
    parameters : infile : file handler of a giant input file
    i : i-th input file
    n : number of articles in each output file
    '''
    counter = {}
    line = infile.readline()
    pgid = -1
    num = 0
    
    while line:
        res = json.loads(line) 
        cgid = res['gid']
        num += 1
        if (cgid != pgid): 
            pgid = cgid
            counter[num] = 1 if num not in counter else (counter[num] + 1)
            num = 0
        line = infile.readline()
    if num > 0: 
        counter[num] = 1 if num not in counter else (counter[num] + 1)
    return Counter(counter)

In [7]:
'''
break-up reuter news files 
'''
with open(infile, 'r', errors='replace') as f:
    breakFile(f)
f.close()

15000
30000
45000
60000
75000
90000
105000
120000
135000
150000
165000
180000
195000
210000
225000
240000
255000
270000
285000
300000
315000
330000
345000
360000
375000
390000
405000
420000
435000
450000
465000
480000
495000
510000
525000
540000
555000
570000
585000
600000
615000
630000
645000
660000
675000
690000
705000
720000
735000
750000
765000
780000
795000
810000
825000
840000
855000
870000
885000
900000
915000
930000
945000
960000
975000
990000
1005000
1020000
1035000
1050000
1065000
1080000
1095000
1110000
1125000
1140000
1155000
1170000
1185000
1200000
1215000
1230000
1245000
1260000
1275000
1290000
1305000
1320000
1335000
1350000
1365000
1380000
1395000
1410000
1425000
1440000
1455000
1470000
1485000
1500000
1515000
1530000
1545000
1560000
1575000
1590000
1605000
1620000
1635000
1650000
1665000
1680000
1695000
1710000
1725000
1740000
1755000
1770000
1785000
1800000
1815000
1830000
1845000
1860000
1875000
1890000
1905000
1920000
1935000
1950000
1965000
1980000
1995000
2010000


In [ ]:
'''
break-up reuter news files 
'''
with open(infile, 'r', errors='replace') as f:
    counter = countFileSize(f)
    print(counter)
f.close()

In [ ]:
plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
labels, values = rankCounter(counter)
indexes = np.arange(len(labels))
width = 1
plt.bar(indexes, values, width)
plt.xticks(indexes + width * 0.5, labels)
plt.show()

In [ ]:
filtered = Counter({key : value for key, value in counter.items() if int(key) < 50})

plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
labels, values = rankCounter(filtered)
indexes = np.arange(len(labels))
width = 1
plt.bar(indexes, values, width)
plt.xticks(indexes + width * 0.5, labels)
plt.show()